In [36]:
%load_ext autoreload
%autoreload 2

from webbrowser import get
import pandas as pd
from sklearn.metrics import accuracy_score, r2_score
from models import *
import logging
from pretty_logger import get_logger
from pathlib import Path


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# define subset (ignore all other data)
ema = [f"Y{i}" for i in range(1, 8, 1)]
physical = [f"P{i}" for i in range(1, 5, 1)]
social = [f"S{i}" for i in range(1, 8, 1)]

In [38]:
df = pd.read_csv(datafile)
df["date"] = pd.to_datetime(df["day_survey"])
df["C"] = df["date"].apply(
    lambda date: (
        COVIDStatus.POST_COVID if date > date_covid else COVIDStatus.PRE_COVID
    )
)

df_head = df.head(5).copy()

df.rename(columns=reverse_ema_dictionary, inplace=True)
df.set_index(["uid", "date"], inplace=True)

subset = ema + physical + social + sleep + demographic + ["C"]
df = df[subset]
df.dropna(inplace=True)

sets_df = pd.read_parquet(sets_file, engine="pyarrow")

In [39]:
skip_treatments = []
skip_outcomes = []

In [40]:
alphas = [0]

for alpha in alphas:
    for index, model_row in sets_df.iterrows():
        already_fitted_sets = []

        covariate_set = CovariateSet(
            row=model_row,
            data=df,
            outcomes_to_skip=skip_outcomes,
            treatments_to_skip=skip_treatments,
        )

        if not covariate_set.valid_set:

            logger.error(
                f"Skipping {covariate_set} (no valid set for "
                f"outcome:{model_row['outcome']}, "
                f"treatment:{model_row['treatment']}, "
                f"set:{model_row['sets']})."
            )
            raise ValueError("Invalid set")
            continue

        already_fitted_sets.append(covariate_set.set_to_fit)

        logger.info(f"Fitting\n{covariate_set!r}")
        logger.info(
            f"Median of {covariate_set.treatment} {df[covariate_set.treatment].median()}"
        )
        wbm = WBLinearModel(
            data=df,
            alpha=alpha,
            treatment=covariate_set.treatment,
            outcome=covariate_set.outcome,
            separating_set=covariate_set.restricted_adjustment_set,
            name=f"row:{index}",
        )
        logger.info(
            f"pre_rsq train={wbm.pre_r_squared[0]}, pre_rsq test={wbm.pre_r_squared[1]}\n"
            f"pre mae test ={wbm.pre_r_squared[2]}\n"
            f"post_rsq train={wbm.post_r_squared[0]}, post_rsq test={wbm.post_r_squared[1]}\n"
        )
        logger.info("-----------------------------------------------------\n")

[2024-05-28 23:08:13.167] - modelslog- INFO N/A- 1968370032.py: 27 [<module>]: Fitting
treatment: D2:race, outcome: Y1:pam, adjustment set={'D1', 'S2', 'S1', 'P3', 'P2', 'P1', 'D3', 'P4', 'D4'}
[2024-05-28 23:08:13.169] - modelslog- INFO N/A- 1968370032.py: 28 [<module>]: Median of D2 1.0
[2024-05-28 23:08:13.287] - modelslog- INFO N/A- 1968370032.py: 39 [<module>]: pre_rsq train=0.05919075256500028, pre_rsq test=0.048916329798104896
pre mae test =-4.581224844531386e-06
post_rsq train=0.06755942602708265, post_rsq test=0.055213106390584854

[2024-05-28 23:08:13.288] - modelslog- INFO N/A- 1968370032.py: 44 [<module>]: -----------------------------------------------------

[2024-05-28 23:08:13.289] - modelslog- INFO N/A- 1968370032.py: 27 [<module>]: Fitting
treatment: D4:cohort year, outcome: Y1:pam, adjustment set={'S7', 'D1', 'S2', 'S6', 'P1', 'D3'}
[2024-05-28 23:08:13.291] - modelslog- INFO N/A- 1968370032.py: 28 [<module>]: Median of D4 1.0
[2024-05-28 23:08:13.399] - modelslog- I